http://www2.cao.ie/points/l8.php

***

In [1]:
#convenient http requests
import requests as rq

# regular expressions
import re

#dates and times
import datetime as dt

# for dataframes import pandas
# v useful to get data from spreadsheets
import pandas as pd

#for downloading files to python
import urllib.request as urlrq



# 2021 points

http://www2.cao.ie/points/l8.php

***


In [2]:
# fetch the CAO points URL
# resp= response from the URL
# rq= request library
# get= get the url (http version)
resp = rq.get ('http://www2.cao.ie/points/l8.php')
#have a quick peek
resp
# resp [200] means html response is in order.  [404] if returns means it is not found .  These are http codes

<Response [200]>

In [3]:
now = dt.datetime.now()


In [4]:
#datetime function has a data structure called datetime
#field for time, teat ,month etc
#dt.datetime.now()
now = dt.datetime.now()

In [5]:
print(now)

2021-11-02 22:33:25.628183


### use the current time as a filename - need to remove colons, decimal points etc

In [6]:
# good programmers use only alphanumeric characters ,dash and dot in file names
#create a filepath for the original data
#use year at start to ensure alphabetic sorted when filed
# below creates the csv file and saves it in a new 'data' folder in active location  ie. \repo\Fun_of..
path = 'data/cao2021_' + now.strftime('%Y%m%d_%H%M%S')  + '.html'

In [7]:
# format now as a string
now.strftime('%Y%m%d_%H%M%S')

'20211102_223325'

### the server uses the wrong encoding fix it before saving

In [8]:
#RESP = RESPONSE
#'resp.text' takes the response and converts it to text- but still has original errors
#thus not best way to save THIS original file
#but resp.text is a typical way to save

In [9]:
original_encoding = resp.encoding
resp.encoding = 'cp1252'

In [10]:
#save the original html file

with open(path, 'w') as f:
    f.write(resp.text)    

In [11]:
resp.text
#\n is a new line (common Unix command)

'<html>\n<BODY>\n<a name="deg"><hr>\n<h3><center> Points Required for Entry to 2021 Level 8 Courses</h3></center><p>\n<A HREF="l76.php">Go to Level 7/6 List of Institutions.</a><p>\n<PRE><b>\n*     Not all on this points score were offered places.\n#     Test / Interview / Portfolio / Audition\nAQA   All qualified applicants\nv     New competition for available places\n \n</PRE>\n<dl>\n<p>\n<dt><a href="#ac8"><b>American College Dublin</b></a>\n<dt><a href="#al8"><b>Athlone Institute of Technology</b></a>\n<dt><a href="#by8"><b>IBAT College Dublin</b></a>\n<dt><a href="#cm8"><b>Marino Institute of Education</b></a>\n<dt><a href="#pc8"><b>Carlow College</b></a>\n<dt><a href="#ct8"><b>CCT College Dublin</b></a>\n<dt><a href="#cr8"><b>Cork Institute of Technology</b></a>\n<dt><a href="#db8"><b>Dublin Business School</b></a>\n<dt><a href="#dc8"><b>Dublin City University</b></a>\n<dt><a href="#dk8"><b>Dundalk Institute of Technology</b></a>\n<dt><a href="#dl8"><b>Dun Laoghaire Institute of 

In [12]:
#how to loop throught the linesv in request. iterate through the lines in request
for line in resp.iter_lines():
    print(line)

b'<html>'
b'<BODY>'
b'<a name="deg"><hr>'
b'<h3><center> Points Required for Entry to 2021 Level 8 Courses</h3></center><p>'
b'<A HREF="l76.php">Go to Level 7/6 List of Institutions.</a><p>'
b'<PRE><b>'
b'*     Not all on this points score were offered places.'
b'#     Test / Interview / Portfolio / Audition'
b'AQA   All qualified applicants'
b'v     New competition for available places'
b' '
b'</PRE>'
b'<dl>'
b'<p>'
b'<dt><a href="#ac8"><b>American College Dublin</b></a>'
b'<dt><a href="#al8"><b>Athlone Institute of Technology</b></a>'
b'<dt><a href="#by8"><b>IBAT College Dublin</b></a>'
b'<dt><a href="#cm8"><b>Marino Institute of Education</b></a>'
b'<dt><a href="#pc8"><b>Carlow College</b></a>'
b'<dt><a href="#ct8"><b>CCT College Dublin</b></a>'
b'<dt><a href="#cr8"><b>Cork Institute of Technology</b></a>'
b'<dt><a href="#db8"><b>Dublin Business School</b></a>'
b'<dt><a href="#dc8"><b>Dublin City University</b></a>'
b'<dt><a href="#dk8"><b>Dundalk Institute of Technology</b></a>'
b'

In [13]:
#compile the regular expression for matching lines
# re= regular expression. Each chsracter is a string but together form an instruction.
# r. = 'raw' used in python. It means don't evaluate backslashes in encountered string literals  i.e. \n \t i.e 
re_course = re.compile(r'([A-Z]{2}[0-9]{3})  (.*)([0-9]{3})(\*?) *')

<br>

### error on the server= way to fix

match only the lines we want -the ones representing courses 
 technically the server says we should decode as:
 
content-type: text/html; charset: iso-8859-1

  but
    -one line uses /x96 which isn't defined in iso-8859-1
    -therefore we used the similiar decoding standard (codec) cp-1252 which is very similar but includes \x96

In [14]:
#for the below if used in assessment should be cleaned up. Check regular expressions are working properly
# if want to save output of this then need a filepath and file format like the original data above

#the filepath for the csv file
path = 'data/cao2021_csv' + now.strftime('%Y%m%d_%H%M%S')  + '.csv'

no_lines = 0
#open the csv file for writing
with open(path, 'w') as f:
   # f.write(resp.text)   
    #loop through the lines of the response content
    for line in resp.iter_lines():
   #decode the line using the encoding not flagged by the response from CAO
        dline = line.decode('cp1252')
        if re_course.fullmatch(dline):
            no_lines = no_lines + 1
            #print(line) no need to print so commented out
            # version 1 using regular expression
            # csv_version = re_course.sub(r'\1,\2,\3,\4', dline) version 1 using re commented out
            #print(csv_version)  no need to print so commented out
            #alternate way than regular expression is to use python in built split function
            # split splits the line
            #(+ ) instructs to do this where two or more spaces
            linesplit = re.split(' +', dline)
            #print(','.join(linesplit))
            #f.write(csv_version + '\n') - saving of old RE way
            #rejoin the substrings with commasin between
            f.write(','.join(linesplit) + '\n')
print(f"total number of lines is {no_lines}.")       

total number of lines is 922.


In [15]:
#old code
#how to loop throught the linesv in request. iterate through the lines in request using a regular expression
#for line in resp.iter_lines():
   # if re.match('[A-Z]{2}[0-9]{3}  .* *(0-9){3} *', line.decode('utf-8') ):
               # print(line)

<br>

## 2020 points

http://www.cao.ie/index.php?page=points&p=2020

***

### download and parse the excel spreadsheet using pandas

In [16]:
#download and parse the excel spreadsheet
df = pd.read_excel ('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx', skiprows = 10)
# however it reads the 1st 10 preamble headings 
# could save file and manually delete 1st 10 rows but better to use pandas functionality

In [17]:
df

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
#spot check of data
# use loc or iloc functionality to check data
df.iloc[12]
#check the final line in the spreadsheet
# reminder 1st 11 rows are heading #excel identifies 1st row as zero
df.iloc[753]
# can also count backwards to last row
df.iloc[-1]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Mechanical and Manufacturing Engineering
COURSE CODE2                                                           WD230
R1 POINTS                                                                253
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      253
EOS Random *                                                             NaN
EOS Mid-point                                                            369
LEVEL                                                                      8
HEI                                        Waterford Institute of Technology
Test/Interview #                                                         NaN

<br>
### save original file 

***

In [19]:
path = 'data/cao2020_' + now.strftime('%Y%m%d_%H%M%S')  + '.xlsx'
now.strftime('%Y%m%d_%H%M%S')
#save the original html file
with open(path, 'w') as f:
    f.write(resp.text)   

In [20]:
#urllib save the original file to disk

urlrq.urlretrieve ('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx', path)

('data/cao2020_20211102_223325.xlsx',
 <http.client.HTTPMessage at 0x22fe14df760>)

<br>
## create a filepath for pandas data


In [21]:
path = 'data/cao2020_' + now.strftime('%Y%m%d_%H%M%S')  + '.csv'
df.to_csv(path)

https://pandas.pydata.org/docs/reference/api/pandas.read_excel.htmlpandas.read_excel <br> function signature in pandas excel webpage: (io, sheet_name=0, header=0, names=None, index_col=None, usecols=None, squeeze=False, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, parse_dates=False, date_parser=None, thousands=None, comment=None, skipfooter=0, convert_float=None, mangle_dupe_cols=True, storage_options=None)¶
skiprows is what we need

## fetch earlier year points data and save in data files

<br>

## 2019 points

http://www.cao.ie/index.php?page=points&p=2019

***

-Note the 2019 and earlier points are in pdf format<br>
Main points inndex url: <br>
https://www.cao.ie/index.php?page=points


<br>
### steps to reproduce the 2019 editable:
****


Manual way of extracting:
1. save pdf in repository
2. open original with MS Word (may need to 'choose other app to open' in dropdown
3. MS Word converts pdf to Word
4. Save file as xxxxx.docx format
5. Resave a copy to edit in repo as xxxxx_edited.docx
6  Delete headers and footers
7. delete preamble on page 1
8. Ctrl 'A', Ctrl 'C' to copy all data in the 21 separate tables
9. Open notepad++ and paste 'Ctrl V'


### manual editing in notepad ++
1. Add HEI to header
2.Select the 'College/Institute' i.e. 'Athlone Institute of Technology'
3. Ctrl X to cut
4. Move cursor to in front of AIT 1st course line
5. Hold down ALT key and drag cursor down to all AIT courses
6. Ctrl V to paste - click 'tab' key to create tabs
7. Delete blank lines
8. Put double quotes around college names as they have commas [select start of string with cursor-alt-type "]
9. Change backticks to apostrophes
10. optional : \t  in extended file 'replace all' with comma: replaces all the tabs with commas <br> however not recommended here as commas elsewhere will confuse. <br> best to leave tabs as delimiters  e.g. in effect its as 'tsv' tab serapated file but can save as csv
11. find \\t replace with \t finds the double tabs and removes them
11. save file

In [22]:
df_2019 = pd.read_csv('data/cao2019_20211102_191031_edited.csv') 

ParserError: Error tokenizing data. C error: Expected 1 fields in line 32, saw 2


In [ ]:
# fetch the CAO points URL.  The url for earlier years takes you to a pdf version
# However the python code to fetch appears to work
resp = rq.get ('http://www2.cao.ie/points/lvl8_19.pdf')
#have a quick peek
resp
# resp [200] means html response is in order.  [404] if returns means it is not found .  These are http codes

In [ ]:
path = 'data/cao2019_' + now.strftime('%Y%m%d_%H%M%S')  + '.html'
now.strftime('%Y%m%d_%H%M%S')
#save the original html file
with open(path, 'w') as f:
    f.write(resp.text)   

#### fetch 2018

In [ ]:
resp = rq.get ('http://www2.cao.ie/points/lvl8_18.pdf')
resp

In [ ]:
path = 'data/cao2018_' + now.strftime('%Y%m%d_%H%M%S')  + '.html'
now.strftime('%Y%m%d_%H%M%S')
#save the original html file
with open(path, 'w') as f:
    f.write(resp.text) 

###How to convert pdf to csv in python- further reading: <br>
-https://stackoverflow.com/questions/49560486/how-to-convert-pdf-to-csv-with-tabula-py

https://www.prepressure.com/library/technology/ascii-binary-hex

http://www.i18nqa.com/debug/table-iso8859-1-vs-windows-1252.html

https://docs.python.org/3/library/datetime.html

In [ ]:
#strftime turns datetime to strings
#strptime turns strings into datetime